# Сравнение OPE (DR) с оракулом
Загружаем результаты синтетического эксперимента и сравниваем оценку Doubly Robust с истинными значениями.

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt

with open('artifacts/summary.json') as f:
    summary = json.load(f)

df = pd.DataFrame({
    'metric': ['accept', 'cltv'],
    'DR': [summary['accept']['dr_B'], summary['cltv']['dr_B']],
    'oracle': [summary['accept']['oracle_B'], summary['cltv']['oracle_B']]
})
df['abs_error'] = (df['DR'] - df['oracle']).abs()
display(df)

df.set_index('metric')[['DR', 'oracle']].plot(kind='bar')
plt.ylabel('value')
plt.title('DR OPE vs Oracle')
plt.show()


Абсолютные ошибки указывают, насколько приближение OPE отличается от истинного эффекта.
При работе на синтетике полезно контролировать эту погрешность, чтобы убедиться в состоятельности метода.